# Importing libraries

In [1]:
#!/usr/bin/env python
import time
import collections
import numpy as np
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
from scipy.signal import find_peaks
import pickle

import sklearn
from sklearn import metrics
from sklearn import neighbors

import pandas as pd
import re
import csv
import os

import paho.mqtt.client as mqtt

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Start MQTT Broker

In [3]:
broker_path = "C:\\Program Files (x86)\\Mosquitto"
os.chdir(broker_path)
print(os.getcwd())
os.system('cmd /k "mosquitto -c mosquitto_conf.conf -v"')

C:\Program Files (x86)\Mosquitto


0

In [4]:
path = "C:\\Users\\ieliz\\Documents\\2021\\FYP\\Stress_Detection_FYP\\python_scripts\\"
os.chdir(path)
print(os.getcwd())

C:\Users\ieliz\Documents\2021\FYP\Stress_Detection_FYP\python_scripts


# Setting up MQTT connections

In [3]:
rec_message = ""

In [4]:
import paho.mqtt.client as mqtt
# declaring variables and callback functions
broker_address="192.168.1.125"

topic="data_reading"

def on_message(client, userdata, message):
    global rec_message
    rec_message = str(message.payload.decode("utf-8"))
    #print(rec_message)
    
# create client instance
# Client constructor: Client(client_id="", clean_session=True, userdata=None, protocol=MQTTv311, transport="tcp")
client = mqtt.Client("ComputerClient")
#print("Created client instance")

# when client receives message, it generates an on_message callback
client.on_message=on_message

# connecting to broker
client.connect(broker_address)
#print("Connected to broker")

0

# Setting up prediction

In [5]:
# Load the pickled model
saved_model_name = "knn_model_hrv_sc_1.sav"

clf = pickle.load(open(saved_model_name, 'rb'))
clf

KNeighborsClassifier(weights='distance')

In [6]:
def find_rmssd(data):
    rr_diff = np.diff(data)
    rr_diff[-1] = 0
    rr_sqdiff = np.power(rr_diff, 2)
    rmssd = np.sqrt(np.mean(rr_sqdiff))
    return rmssd

In [7]:
def predict_live(model, dataframe, window_size):
    
    # grab moving window of data
    size = len(dataframe.index)
    if size >= window_size:
        df = dataframe.iloc[-(window_size+1):].copy()
    else:
        df = dataframe.copy()
            
    # min-max norm
    min_val_gsr = min(df['Conductance (uS)'])
    max_val_gsr = max(df['Conductance (uS)'])
    scaling_gsr = max_val_gsr-min_val_gsr
    df.loc[:, 'Normalised_GSR'] = (df.loc[:, 'Conductance (uS)']- min_val_gsr)/scaling_gsr 
    
    min_val_hr = min(df["Heart_Rate"])+np.random.rand()/10 #random number avoids dividing by 0
    max_val_hr = max(df["Heart_Rate"])+np.random.rand()/10
    scaling_hr = max_val_hr-min_val_hr
    df.loc[:, "Normalised_HR"] = (df.loc[:, "Heart_Rate"]- min_val_hr)/scaling_hr 
           
    HR_data = df["Heart_Rate"].to_numpy()
    GSR_data = df["Conductance (uS)"].to_numpy()
    HR_norm_data = df["Normalised_HR"].to_numpy()
    GSR_norm_data = df["Normalised_GSR"].to_numpy()
    HRV_data = df["HRV"].to_numpy()
                
    hr_mean = np.mean(HR_norm_data)       # hr mean
    hr_std = np.std(HR_norm_data)         # hr std
    hr_diff = max(HR_data) - min(HR_data) # hr diff
    
    hrv_mean = np.mean(HRV_data)          # hrv mean
    hrv_std = np.std(HRV_data)            # hrv std
    hrv_rmssd = find_rmssd(HRV_data)      # hrv rmssd
    
    gsr_mean = np.mean(GSR_norm_data)     # gsr mean
    gsr_std = np.std(GSR_norm_data)       # gsr std
        
    # find gsr peaks
    curr_peaks_ind, _ = find_peaks(GSR_norm_data)
    curr_peaks = GSR_norm_data[curr_peaks_ind]
    
    gsr_num_peaks = len(curr_peaks)       # gsr num peaks
    gsr_rate_peaks = gsr_num_peaks/len(GSR_norm_data)   # gsr rate peaks
    gsr_mean_peaks = np.mean(curr_peaks)  # gsr mean peaks
    gst_std_peaks = np.std(curr_peaks)    # gsr std peaks
            
#     print([hr_mean, hr_std, hr_diff, hrv_mean, hrv_std, hrv_rmssd, gsr_mean, gsr_std, gsr_rate_peaks, gsr_mean_peaks, gst_std_peaks])
    prediction = clf.predict(np.c_[hr_mean, hr_std, hr_diff, hrv_mean, hrv_std, hrv_rmssd, gsr_mean, gsr_std, gsr_rate_peaks, gsr_mean_peaks, gst_std_peaks])
        
    return prediction

# Reading in data

In [8]:
print("Recording"+ " Started")
# run a loop otherwise, miss callbacks
client.loop_start()

# subscribing to topic
client.subscribe(topic)
print("Subscribed to topic", topic)

Recording Started
Subscribed to topic data_reading


In [ ]:
while True:
    print(rec_message)
    time.sleep(2)







BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510, BATT=371.51
BPM=0, GSR=510

In [12]:
r = re.compile("(?<==)([0-9]+)")

idx = 0
hr_data = []
hrv_data = []
gsr_data = []
resistance_data = []
conductance_data = []
prediction_data = []

df_predict = pd.DataFrame(columns = ['Time (hr:min:sec)', 'Time (s)', 'GSR', 'Resistance', 'Conductance (uS)', 'Heart_Rate', 'HRV', 'Section'])

tempname = "plot_test"
filename = tempname + ".csv"
f = open(filename, "w", newline='', encoding='utf-8')
writer = csv.writer(f, delimiter=',', quotechar='"')
writer.writerow(['Time (hr:min:sec)', 'Time (s)', 'GSR', 'Resistance', 'Conductance (uS)', 'Heart_Rate', 'HRV', 'Section', 'Stressed'])

92

In [13]:
class Worker(QtCore.QRunnable):
    '''
    Worker Thread
    
    Inherits from QRunnable to handler worker thread setup, signals and wrap-up.
    
    :param args: Arguments to make available to the run code
    :param kwargs: Keywords arguments to make available to the run code

    Container for work being performed
    '''
    
    def __init__(self, *args, **kwargs):
        super(Worker, self).__init__()
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()
    
    @QtCore.pyqtSlot()
    def run(self):
#         print("Thread start")
        global r, rec_message, idx, hr_data, hrv_data, resistance_data, conductance_data, prediction_data, df_predict, clf, predict_live, tempname, writer
        try:
            idx += 1

            self.match = re.findall(r, rec_message)
            self.hr = int(self.match[0])
            self.gsr = int(self.match[1])
            self.batt = int(self.match[2])/100 # convert back to float

            hr_data.append(self.hr)

            self.hrv = 60000/self.hr
            hrv_data.append(self.hrv)

            self.resistance = (2**10 + 2*self.gsr)/(2**9-self.gsr)*10000
            gsr_data.append(self.gsr)
            resistance_data.append(self.resistance)

            self.conductance = 1/self.resistance*1000000
            conductance_data.append(self.conductance)

            t = time.strftime("%H:%M:%S", time.localtime())
            df_predict.loc[idx] = [t, idx, self.gsr, self.resistance, self.conductance, self.hr, self.hrv, tempname]
            #print(df_predict.loc[idx])

            if len(hr_data) >= 10:
                self.prediction = predict_live(clf, df_predict, 30)
                self.pred = self.prediction[0]
                prediction_data.append(self.pred)
                if self.pred >= 5:
                    self.pred_label = "Stressed"
                else:
                    self.pred_label = "Not Stressed"
#                 print(self.batt, self.hr, self.gsr, self.pred, self.pred_label)

                writer.writerow([t, idx, self.gsr, self.resistance, self.conductance, self.hr, self.hrv, tempname, prediction_data[-1]])

                self.signals.result.emit([self.hr, self.conductance, self.pred, self.pred_label, self.batt])

        except:
            print("Oh no")
        
#         print("Thread complete")

In [14]:
class WorkerSignals(QtCore.QObject):
    '''
    Defines the signals available from a running worker thread.
    '''
    
    result = QtCore.pyqtSignal(object)

In [15]:
class MainGUI():
    def __init__(self):
        # Initialise window name and size
#         pg.setConfigOption('background', '#E1E1E1')
        pg.setConfigOption('background', 'w')
        pg.setConfigOption('foreground', 'k')
        self.win = pg.GraphicsLayoutWidget(show=True, title="Stress Detection")
        self.win.title="Stress Detection"
        self.win.resize(1000,600)
        self.win.setWindowTitle('Stress Detection')
        
        # Initialise counters and variables
        self.ptr = 0
        self.frame = 60
        
        self.hr = 0
        self.gsr = 0
        self.data_hr = []
        self.data_gsr = []
        self.batt = 0
        self.stress_lev = 0
        self.stress_lab = ""
        
        # GUI - First row
        self.p1 = self.win.addPlot(title="Heart Rate")
        self.curve_hr = self.p1.plot(pen=pg.mkPen('r', width=5))
        self.p1.setLabel('left', "Heart Rate", units='BPM')
        self.p1.setLabel('bottom', "Time", units='s')
        
        self.label_hr = pg.LabelItem(justify='center')
        self.win.addItem(self.label_hr)
        
        # GUI - Second Row
        self.win.nextRow()

        self.p2 = self.win.addPlot(title="Galvanic Skin Response")
        self.curve_gsr = self.p2.plot(pen=pg.mkPen('g', width=5))
        self.p2.setLabel('left', "GSR", units='uC')
        self.p2.setLabel('bottom', "Time", units='s')

        self.label_gsr = pg.LabelItem(justify='center')
        self.win.addItem(self.label_gsr)

        # GUI - Third Row, RHS
        self.win.nextRow()
        self.win.nextCol()
        
        self.label_stress_lab = pg.LabelItem(justify='left')
        self.win.addItem(self.label_stress_lab)
        
        # GUI - Fourth Row, LHS
        self.win.nextRow()
        
        self.label_battery = pg.LabelItem(justify='left')
        self.win.addItem(self.label_battery)
        
        # GUI - Fourth Row, RHS
        self.label_stress_lev = pg.LabelItem(justify='left')
        self.win.addItem(self.label_stress_lev)

        # Start Threadpool, allowing threading
        self.threadpool = QtCore.QThreadPool()
        print("Multithreading with maximum %d threads" % self.threadpool.maxThreadCount())
        
        # Update GUI every second
        # Call Update function every second, using 1000msec timer
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update)
        self.timer.start(1000)
        
    def print_output(self, s):
        self.hr = s[0]
        self.gsr = s[1]
        self.data_hr.append(self.hr)
        self.data_gsr.append(self.gsr)
        self.stress_lev = s[2]
        self.stress_lab = s[3]
        self.batt = s[4]
        print(s)
        
    def update(self):
        # Start worker to update GUI and print outputs
        worker = Worker() # Worker reads MQTT data, processes it and predicts; emitting output as signal
        worker.signals.result.connect(self.print_output) #Receives emitted output and calls print_output
        self.threadpool.start(worker)
        
        # Update display data in GUI
        if len(self.data_hr) > self.frame:
            self.curve_hr.setData(self.data_hr[len(self.data_hr)-self.frame:])
            self.curve_gsr.setData(self.data_gsr[len(self.data_hr)-self.frame:])
        else:
            self.curve_hr.setData(self.data_hr[:])
            self.curve_gsr.setData(self.data_gsr[:])
            
#         global data_hr, data_gsr
#         self.curve_hr.setData(data_hr[self.ptr:self.ptr+self.frame])
#         self.curve_gsr.setData(data_gsr[self.ptr:self.ptr+self.frame])
#         self.curve_hr.setData([60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70])
#         self.curve_gsr.setData([8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])
        # First set graphics Y range
        #if self.ptr == 0:
        #    self.p1.setYRange(50, 120)
#             self.p2.setYRange(1, 14)
        self.ptr += 1
        
        # Update display text in GUI
        self.label_hr.setText("HR: "+str(self.hr)+"bpm", size='16pt')
        self.label_gsr.setText("GSR: "+(str(self.gsr))[:5]+"uC", size='16pt')
        self.label_battery.setText("Batt: "+str(self.batt)+"V", size='12pt')
        self.label_stress_lab.setText(str(self.stress_lab), size='16pt')
        self.label_stress_lev.setText("Stress: "+str(self.stress_lev)+"/10", size='12pt')


In [16]:
app = pg.mkQApp("Stress Detection")
pg.setConfigOptions(antialias=True)
window = MainGUI()

Multithreading with maximum 4 threads


In [17]:
app.exec()

Oh no
Oh no
Oh no
Oh no
Oh no
Oh no
Oh no
Oh no
Oh no
Oh no
[73, 1.5609264853977847, 4, 'Not Stressed', 3.68]
[73, 2.620760534429599, 4, 'Not Stressed', 3.69]
[73, 2.3517382413087935, 4, 'Not Stressed', 3.68]
[73, 2.2448979591836733, 4, 'Not Stressed', 3.69]
[73, 2.1916411824668707, 4, 'Not Stressed', 3.69]
[73, 2.1916411824668707, 4, 'Not Stressed', 3.68]
[73, 2.138492871690428, 4, 'Not Stressed', 3.69]
[73, 2.138492871690428, 4, 'Not Stressed', 3.69]
[73, 2.138492871690428, 4, 'Not Stressed', 3.69]
[73, 2.1916411824668707, 4, 'Not Stressed', 3.67]
[75, 2.1916411824668707, 4, 'Not Stressed', 3.69]
[74, 2.138492871690428, 4, 'Not Stressed', 3.69]
[70, 2.138492871690428, 4, 'Not Stressed', 3.69]
[71, 2.1916411824668707, 4, 'Not Stressed', 3.69]
[71, 2.1916411824668707, 4, 'Not Stressed', 3.69]
[71, 2.1916411824668707, 4, 'Not Stressed', 3.69]
[71, 2.1916411824668707, 4, 'Not Stressed', 3.69]
[71, 2.1916411824668707, 4, 'Not Stressed', 3.69]
[71, 2.1916411824668707, 4, 'Not Stressed', 3.

0

In [ ]:
f.close()